## Forcing Phase Transitions in cs5

In [1]:
# load packages I need
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import scipy.stats as sts
import scipy.integrate
import scipy.interpolate
import re

In [2]:
# next we sample a random length in density

def sample_n_interval(num_points, n_start, max_n):
    """
    Function to create an array of randomly sampled density interval in range (n_start, max_n)

    Inputs:
    num_points: int defining number of samples taken
    n_start: first point in returned density array
    max_n: last point in array

    Outputs:
    n_sort: np array of length = num_points + 2 of density points ordered from least to greatest
    """

    # sample random points in density
    epsilon = 1e-17
    loc_n = n_start + epsilon  # to guarentee we don't randomly pull n_start
    scale_n = max_n - n_start - epsilon
    sample_n = sts.uniform.rvs(loc=loc_n, scale=scale_n, size=num_points)
    # order these points
    n_sort = np.sort(sample_n)

    return n_sort


In [13]:
def extend_cs(n_step, ns, starts, cs2_func):
    """
    Function to extend the EOS by using linear segments in speed of sound

    Inputs:
    n_step: int defining step size in density
    ns: array containing the densities where segments begin/end
    starts: array containing the starting values for the EOS (density, pressure, energy)
    cs2_func: linear interpolation of randomly sampled speed of sound values

    Outputs:
    EOS_ex: array of shape (size, 3) containing the values for EOS extension
    """

    size = int((ns[-1] - starts[0]) / n_step)

    # initialize array
    EOS_ex = np.zeros((size, 3))
    # set starting values at n = 2n0
    EOS_ex[0, 0] = starts[0]
    EOS_ex[0, 1] = starts[1]
    EOS_ex[0, 2] = starts[2]

    for k in range(size - 1):
        # n_i+1
        EOS_ex[k + 1, 0] = EOS_ex[k, 0] + n_step
        # p_i+1
        if cs2_func(EOS_ex[k, 0]) > 1:
            EOS_ex[k + 1, 1] = EOS_ex[k, 1] + n_step * ((EOS_ex[k, 1] + EOS_ex[k, 2]) / EOS_ex[k, 0])
        else:
            EOS_ex[k + 1, 1] = EOS_ex[k, 1] + n_step * (cs2_func(EOS_ex[k, 0])) * (
                        (EOS_ex[k, 1] + EOS_ex[k, 2]) / EOS_ex[k, 0])
        # e_i+1
        EOS_ex[k + 1, 2] = EOS_ex[k, 2] + n_step * ((EOS_ex[k, 1] + EOS_ex[k, 2]) / EOS_ex[k, 0])

    return EOS_ex

def stitch_EOS(small_EOS, EOS_ex):
    """
    Function to stitch EOS extension to original EOS

    Inputs:
    small_EOS: array containing original EOS
    EOS_ex: array containing EOS extension

    Outputs:
    tot_EOS: array containing total EOS
    """
    # get relevant sizes
    size_smol = small_EOS.shape[0] - 1  # -1 becuase we don't want last duplicated entry
    size_ex = EOS_ex.shape[0]

    # initialize array
    tot_EOS = np.zeros((size_smol + size_ex, small_EOS.shape[1]))

    tot_EOS[:size_smol, :] = small_EOS[:size_smol, :]
    tot_EOS[size_smol:, 0] = EOS_ex[:, 0]
    tot_EOS[size_smol:, 1] = EOS_ex[:, 1]
    tot_EOS[size_smol:, 2] = EOS_ex[:, 2]

    return tot_EOS

In [4]:
def EOS_params(EOS_file, nsamp):

    EOS = np.loadtxt(EOS_file)
    with open(EOS_file, 'r') as f:
        header = f.readline()
        
    cs, ns = np.zeros(nsamp), np.zeros(nsamp)
    
    for j in range(nsamp):
        if j+1 > nsamp:
            print("variable which_n must be leq nsamp")
        elif j+1 == 1:
            pattern_c = "cs\s*=\[(\d+\.\d+).*\]"
            pattern_n = "ns\s*=\[(\d+\.\d+).*\]"
        elif j+1 == 2:
            pattern_c = "cs\s*=\s*\[\d+\.\d+\s+(\d+\.\d+).*\]"
            pattern_n = "ns\s*=\s*\[\d+\.\d+\s+(\d+\.\d+).*\]"
        elif j+1 == 3:
            pattern_c = "cs\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
            pattern_n = "ns\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
        elif j+1 == 4:
            pattern_c = "cs\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
            pattern_n = "ns\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
        elif j+1 == 5:
            pattern_c = "cs\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
            pattern_n = "ns\s*=\s*\[\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+(\d+\.\d+).*\]"
        else:
            print("something went wrong")
            
        p_c, p_n = re.compile(pattern_c), re.compile(pattern_n)
        
        if type(p_c.search(header)) != type(None):       
            cs[j] = float(p_c.search(header).group(1))
            
        if type(p_n.search(header)) != type(None):
            ns[j] = float(p_n.search(header).group(1))

    return cs, ns

In [15]:
# first we need to load the EoS

datapath = os.getcwd() + "/data/"
EOSdir_name = "cs5EOS/"

i = 300

old_file = datapath+EOSdir_name+str(int(i))+'.dat'
EOS_old = np.loadtxt(old_file)
with open(old_file, 'r') as f:
    header = f.readline()
    
EOS_start = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()
    
n0 = 0.16 #MeV/fm3
max_n = 12*n0
n_step=1e-3

interval = sample_n_interval(2,2*n0,max_n)

old_cs, old_ns = EOS_params(old_file, 5)
print(old_cs, old_ns)

above_interval = old_ns > interval[-1]
below_interval = old_ns < interval[0]
inside_interval = np.logical_and(old_ns < interval[-1], old_ns > interval[0])

almost_full_n = np.append(old_ns[below_interval], interval)
nearly_ns = np.append(almost_full_n, old_ns[above_interval])
ns = np.append(nearly_ns, max_n)

almost_full_c = np.append(old_cs[below_interval], 1e-3*np.ones(2))
nearly_cs = np.append(almost_full_c, old_cs[above_interval])
cs = np.append(nearly_cs, old_cs[-1])

cs2_func = scipy.interpolate.interp1d(ns, cs ** 2)

starts = EOS_start[-1,:]
EOS_ex = extend_cs(n_step, ns, starts, cs2_func)

EOS = stitch_EOS(EOS_start, EOS_ex)

[0.36540786 0.62705655 0.7701586  0.81049303 0.35597752] [0.32       0.38570012 0.58083858 1.84606927 1.86413083]


In [ ]:
print(cs, ns, max_n)

In [ ]:
n = EOS[:,0]
p = EOS[:,1]
e = EOS[:,2]

cs2 = np.zeros(p.shape[0]-1)

for j in range(p.shape[0]-1):
    cs2[j] = (p[j+1]-p[j])/(e[j+1]-e[j])

fig, ax = plt.subplots(1,2, figsize=(16,8))

ax[0].plot(e, p, '-', label="cs5MRL_"+str(i)+" w/ forced phase transistion")
ax[0].set_xlabel('Energy Density (MeV/fm$^3$)')
ax[0].set_ylabel('Pressure (MeV/fm$^3$)')

cs_eq1_EOS = np.loadtxt(datapath+"cs_eq1EOS")
ax[0].plot(cs_eq1_EOS[:,2], cs_eq1_EOS[:,1], color='black', label="cs=1")

cs_eq1 = np.zeros(cs_eq1_EOS.shape[0]-1)
for j in range(cs_eq1_EOS.shape[0]-1):
    cs_eq1[j] = (cs_eq1_EOS[j+1,1]-cs_eq1_EOS[j,1])/(cs_eq1_EOS[j+1,2]-cs_eq1_EOS[j,2])

ax[1].plot(e[:p.shape[0]-1], cs2, label=str(i))
ax[1].plot(cs_eq1_EOS[:cs_eq1_EOS.shape[0]-1,2], cs_eq1, color='black', label="cs=1")
ax[1].hlines(1, xmin=e[0], xmax=e[-2], color='r')
ax[1].set_ylabel('Speed of Sound')
ax[1].set_xlabel('Energy Density (MeV/fm$^3$)')

In [16]:
def extend_w_phase_transition(EOS_start, old_file):
    n0 = 0.16 #MeV/fm3
    max_n = 12*n0
    n_step = 1e-3

    interval = sample_n_interval(2,2*n0,max_n)

    old_cs, old_ns = EOS_params(old_file, 5)

    above_interval = old_ns > interval[-1]
    below_interval = old_ns < interval[0]
    inside_interval = np.logical_and(old_ns < interval[-1], old_ns > interval[0])

    almost_full_n = np.append(old_ns[below_interval], interval)
    nearly_ns = np.append(almost_full_n, old_ns[above_interval])
    ns = np.append(nearly_ns, max_n)

    almost_full_c = np.append(old_cs[below_interval], 1e-2*np.ones(2))
    nearly_cs = np.append(almost_full_c, old_cs[above_interval])
    cs = np.append(nearly_cs, old_cs[-1])

    cs2_func = scipy.interpolate.interp1d(ns, cs ** 2)

    starts = EOS_start[-1,:]
    EOS_ex = extend_cs(n_step, ns, starts, cs2_func)

    EOS = stitch_EOS(EOS_start, EOS_ex)
    
    return EOS, ns, cs

In [7]:
def progress_statement(i, total):
    if int(i*100/total) == 25 and int((i-1)*100/total) != 25:
        print("The run is 25% complete")
        done_25=True
    elif int(i*100/total) == 50 and int((i-1)*100/total) != 50:
        print("The run is 50% complete")
        done_50=True
    elif int(i*100/total) == 75 and int((i-1)*100/total) != 75:
        print("The run is 75% complete")
        done_75=True
    
def filter_MRL(MRL_table, m_ub=5, r_lb=7):
    raw_mass = MRL_table[:,0]
    raw_radius = MRL_table[:,1]
    raw_Lambda = MRL_table[:,2]
    # create boolean arrays to test if points are good
    m2big = raw_mass < m_ub
    r2small = raw_radius > r_lb
    # the bool array of points we will keep
    keep = np.logical_and(m2big, r2small)
    # define new arrays we will keep, this filters out erronious points in array
    radius = raw_radius[keep]
    mass = raw_mass[keep]
    Lambda = raw_Lambda[keep]
    
    leng = len(radius) # get number of physical points
    MRL = np.zeros((leng, 3)) # initialize MRL table
    MRL[:,0], MRL[:,1], MRL[:,2] = mass, radius, Lambda # put into table
    
    return MRL

def exclude_by_mass(MRL, maxm_lb=1.8, maxm_ub=4.1):
    maxm = np.max(MRL[:,0])
    if maxm < maxm_ub and maxm > maxm_lb:
        MRL_passes = True
    else:
        MRL_passes = False
    return MRL_passes

def mkdirs(nsamp, ext_type, newsavename, datapath):
    if newsavename==None:
        EOSdir_name = ext_type+str(nsamp)+'EOS'
        MRLdir_name = ext_type+str(nsamp)+'MRL'
    else:
        EOSdir_name = ext_type+str(nsamp)+'EOS'+newsavename
        MRLdir_name = ext_type+str(nsamp)+'MRL'+newsavename
    
    if not(EOSdir_name in os.listdir(datapath)) and not(MRLdir_name in os.listdir(datapath)):
        os.makedirs(datapath+EOSdir_name)
        os.makedirs(datapath+MRLdir_name)
        
    return EOSdir_name, MRLdir_name

def get_num_eos(dirpath):
    numlist = []
    for file in os.listdir(dirpath):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
    
    if len(numlist) == 0:
        num_eos = 0
    else:
        num_eos = max(numlist)
        
    return num_eos

In [9]:
ext_type = 'cs'
nsamp = 5
newsavename = "*"

if newsavename == None:
    EOSdir_name = ext_type+str(nsamp)+'EOS'
    MRLdir_name = ext_type+str(nsamp)+'MRL'
    
else:
    EOSdir_name = ext_type+str(nsamp)+'EOS'+newsavename
    MRLdir_name = ext_type+str(nsamp)+'MRL'+newsavename

datapath = os.getcwd() + "/data/"

numlist = []
num_eos, num_mrl = 0, 0
for file in os.listdir(datapath+EOSdir_name):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
            num_eos += 1
            
for file in os.listdir(datapath+MRLdir_name):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
            num_mrl += 1

print(num_mrl)

255


In [19]:
import TOVsolver
datapath = os.getcwd() + "/data/"
oldEOSdir_name = "cs5EOS/"

# EOS from which we extend
EOS_start = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()

num_eos = get_num_eos(datapath+oldEOSdir_name)

# make new directories with proper names
ext_type = "cs"
nsamp = 5
newsavename = "*"
EOSdir_name, MRLdir_name = mkdirs(nsamp, ext_type, newsavename, datapath)

skip = 937

for i in range(num_eos):
    # get old file parameters
    old_file = datapath+oldEOSdir_name+str(int(i)+skip)+'.dat'
    EOS_old = np.loadtxt(old_file)
    with open(old_file, 'r') as f:
        header = f.readline()

    MRL_inrange=False
    while MRL_inrange==False:
        EOS, ns, cs = extend_w_phase_transition(EOS_start, old_file)
        param_string = "ns =" + str(ns) + ' cs =' + str(cs)

        MRL_table = TOVsolver.solve(EOS, size=100) #solve tov
        # filter out erronious points
        MRL = filter_MRL(MRL_table)
        #check if maximum mass is realistic
        MRL_inrange = exclude_by_mass(MRL, maxm_lb=1.8)

    EOSname = datapath + EOSdir_name + '/' + str(i+skip) + '.dat' # make names for file
    MRLname = datapath + MRLdir_name + '/' + str(i+skip) + '.dat'
    np.savetxt(EOSname, EOS, header=param_string) # save files
    np.savetxt(MRLname, MRL)
#     print("saving file...")
    progress_statement(i, num_eos)

KeyboardInterrupt: 

In [20]:
print(i+skip)

960
